In [1]:
from helpers.data_preparation import * 
from helpers.nlp_functions import *
from helpers.time_series_functions import *

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [ ]:
stock_list = stockListFromURL(market="usa")
stock_data = stockDataFromYf(stock_list)
stock_list = filterStocksByVolume(stock_data, largest_n=200)
price_data = stock_data["Close"][stock_list]

threshold = 0.3 * len(price_data)
price_data = price_data.dropna(thresh=len(price_data) - threshold, axis=1)
price_data = price_data.ffill()
price_data = price_data.bfill()

stock_list = list(price_data.columns)
stock2id = stocks2id(stock_list)
stock_data = stockDataFromYf(stock_list)

textual_data = fetchTextualInformation(stock_list, stock2id)

In [84]:
pd.DataFrame(textual_data).T.sample(5)

,ticker_id,code,name,sector,industry,description
DXCM,182,DXCM,"DexCom, Inc.",Healthcare,Medical Devices,"DexCom, Inc., a medical device company, focuse..."
PM,114,PM,Philip Morris International Inc,Consumer Defensive,Tobacco,Philip Morris International Inc. operates as a...
AMD,3,AMD,"Advanced Micro Devices, Inc.",Technology,Semiconductors,"Advanced Micro Devices, Inc. operates as a sem..."
FI,164,FI,"Fiserv, Inc.",Technology,Information Technology Services,"Fiserv, Inc., together with its subsidiaries, ..."
NCLH,27,NCLH,Norwegian Cruise Line Holdings,Consumer Cyclical,Travel Services,"Norwegian Cruise Line Holdings Ltd., together ..."


In [85]:
stock_list = textual_data.keys()

In [86]:
len(stock_list)

200

In [62]:
tokenizer, nlp_model = load_model()
description_feature_vectors = {}

for i, key in enumerate(list(textual_data.keys())):
    desc = textual_data[key]["description"]
    feature_vector = extract_features(text=desc, tokenizer=tokenizer, model=nlp_model)
    description_feature_vectors[i] = feature_vector

title = "usa"
with open(f'cold_data/description_feature_vectors{title}.pkl', 'wb') as f:
    pickle.dump(description_feature_vectors, f)

price_data = price_data[list(textual_data.keys())]
sim_metrics = ["pearson", "dtw", "dist_corr", "euclidean"]
for metric in sim_metrics:
    ts_sim = calculate_historical_ts_similarities(price_data, start_date="2023-01-01", end_date="2023-06-01", 
                                                  similarity_metric=metric, path=f"range_similarities_{title}.pkl")

    with open(f'calculated_data/historical_ts_{title}_{metric}.pkl', 'wb') as f:
        pickle.dump(ts_sim, f)

100%|██████████| 237/237 [01:58<00:00,  2.01it/s]


In [ ]:
price_data["Date"] = stock_data.index

price_data.to_csv(f"cold_data/stock_data{title}.csv", index=False)
pd.DataFrame(textual_data).to_csv(f"cold_data/textual_information{title}.csv", index=False)

In [80]:
s = pd.DataFrame(columns=list(price_data.drop("Date", axis=1).columns))
s.loc[0] = (list(stocks2id(stock_list).values()))
s.to_csv(f"cold_data/stock2id{title}.csv", index=False)